In [1]:
# Import the libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
import pandas_datareader.data as web
import yfinance as yf

# overide default yfinance download method
yf.pdr_override()
# set backend of pandas to plotly
pd.options.plotting.backend = "plotly"

In [2]:
class OptionsData:

    def __init__(self, ticker: str, expirey: dt.date, option_type: str):
        self.ticker = ticker
        self.expirey = expirey
        self.option_type = option_type
        self.options_data = self.get_options_data()

    
    def get_options_data(self) -> pd.DataFrame:
        """ 
        This function will return a pandas dataframe of the options data for a given ticker, 
        expirey date, and option type. It will return the closest expirey date to the given date
        
        Parameters
        ----------
        ticker : str
            The ticker of the stock you want to get the options data for
        date : datetime.date
            The date you want to get the options data for
        type : str
            The type of option you want to get the data for. Must be either 'call' or 'put'
        
        Returns
        -------
        data : pd.DataFrame
            A pandas dataframe of the options data for the given ticker, expirey date, and option type
        """
        try:    
            options = web.YahooOptions(self.ticker)
            options.headers = {'User-Agent': 'Firefox'}
        except:
            print("Error: Could not get options data for ticker: ", self.ticker)
            return None
    
        # Get closest expirey date to date and reset self.expirey to
        expirey_new = min(options._get_expiry_dates(), key=lambda x: abs(x - self.expirey))
        if self.expirey != expirey_new:
            print("#---------------------#")
            print("Closest expirey date to", self.expirey, "is", expirey_new)
            print("Using expirey date:", expirey_new)
            print("#---------------------#")
            self.expirey = expirey_new
        

        # Get the call data
        if type == "call":
            data = options.get_call_data(expiry=self.expirey)
        elif type == "put":
            data = options.get_put_data(expiry=self.expirey)
        else:
            raise ValueError("option_type must be either 'call' or 'put'")

        # The JSON data is the actual useful data that we want
        data.reset_index(inplace=True)
        data.set_index("Strike", inplace=True)
        data = data.JSON.apply(pd.Series)

        #Drop strike column since it is now the index
        data.drop("strike", axis=1, inplace=True)

        return data

    def get_underlying_data(self, start: dt.date, end: dt.date, interval: str = "1d") -> pd.DataFrame:
        """ 
        This function will return a pandas dataframe of the underlying data for a given ticker, 
        start date, end date, and interval
        
        Parameters
        ----------
        ticker : str
            The ticker of the stock you want to get the options data for
        start : datetime.date
            The start date you want to get the underlying data for
        end : datetime.date
            The end date you want to get the underlying data for
        interval : str
            The interval you want to get the underlying data for. Must be either '1d', '1wk', '1mo'
        
        Returns
        -------
        data : pd.DataFrame
            A pandas dataframe of the underlying data for the given ticker, start date, end date, and interval
        """
        try:
            print("#---------------------#")
            print("Getting underlying data for ticker: ", self.ticker)
            print("#---------------------#")
            data = web.get_data_yahoo(self.ticker, start, end, interval=interval)
        except:
            print("Error: Could not get underlying data for ticker: ", self.ticker)
            return None
        
        return data
    

In [7]:
ticker = "GS"
date = dt.date.today() + dt.timedelta(days=1)
type = "call"

data = OptionsData(ticker, date, type)
options_df = data.options_data
stock_df = data.get_underlying_data(dt.date(2021, 12, 31), dt.date.today(),  "1d")

#---------------------#
Closest expirey date to 2023-04-05 is 2023-04-06
Using expirey date: 2023-04-06
#---------------------#
#---------------------#
Getting underlying data for ticker:  GS
#---------------------#
[*********************100%***********************]  1 of 1 completed


In [8]:
# Plot strike vs bid where data point is coloured by in the money
# in the money true = green, false = red

colors = ['green' if val else 'red' for val in options_df.inTheMoney]

fig = go.Figure()
fig.add_trace(go.Scatter(x=options_df.index, y=options_df.ask, mode='markers', marker_color=colors))
fig.add_trace(go.Scatter(x=stock_df.index, y=stock_df.Close, mode='lines', name='Close'))
fig.update_layout(title="Strike vs Bid", xaxis_title="Strike", yaxis_title="Bid")

fig.show()

In [9]:
# plot implied volatility vs strike
fig = go.Figure()
fig.add_trace(go.Scatter(x=options_df.index, y=options_df.impliedVolatility, mode='lines'))

fig.update_layout(title="Strike vs Implied Volatility", xaxis_title="Strike", yaxis_title="Implied Volatility")
fig.show()

In [10]:
stock_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-31,385.000000,387.000000,380.839996,382.549988,370.066833,1601300
2022-01-03,389.000000,399.299988,387.339996,395.329987,382.429840,3334300
2022-01-04,402.230011,410.279999,401.179993,407.480011,394.183380,3916400
2022-01-05,409.799988,412.660004,397.890015,398.630005,385.622162,3469600
2022-01-06,401.000000,401.200012,386.429993,396.929993,383.977600,2960900
...,...,...,...,...,...,...
2023-03-28,319.089996,320.609985,316.119995,318.440002,318.440002,1224300
2023-03-29,322.500000,322.709991,318.679993,320.720001,320.720001,1475900
2023-03-30,324.559998,324.869995,319.880005,321.140015,321.140015,1456300


In [338]:
# Define the ticker symbol for the repo rate
ticker = 'DPRIME'
start_date = '2001-01-01'
end_date = dt.date.today().strftime("%Y-%m-%d")

cash_rate= web.FredReader(symbols=['DFF', 'DFEDTAR', 'DFEDTARL', 'DFEDTARU'], start=start_date, end=end_date).read()

In [339]:
# Plot the repo rate as a line chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=cash_rate.index, y=cash_rate.DFF, mode='lines', name='Cash Rate'))
fig.add_trace(go.Scatter(x=cash_rate.index, y=cash_rate.DFEDTAR, mode='lines', name='Target Rate'))
fig.add_trace(go.Scatter(x=cash_rate.index, y=cash_rate.DFEDTARL, mode='lines', name='Target Rate Lower Bound'))
fig.add_trace(go.Scatter(x=cash_rate.index, y=cash_rate.DFEDTARU, mode='lines', name='Target Rate Upper Bound'))
fig.update_layout(title="Cash Rate", xaxis_title="Date", yaxis_title="Rate")
fig.show()

In [340]:
ticker = "BKFTTLA641N"
failures = web.FredReader(symbols=ticker, start=start_date, end=end_date).read()



In [342]:
# Plot failures as a line chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=failures.index, y=failures.BKFTTLA641N, mode='lines', name='Failures'))
fig.update_layout(title="Failures", xaxis_title="Date", yaxis_title="Failures")
fig.show()